In [44]:
from bs4 import BeautifulSoup
import requests
import json
from collections import defaultdict
import re
import pandas as pd
import random
import os



In [46]:
import bs4
print(bs4.__version__)

4.12.3


In [48]:
def get_champ_dict(champ_ref):
    download_folder = "downloaded_images"
    url = f"https://tftactics.gg{champ_ref}/"
    res = requests.get(url)
    comm = re.compile("<!--|-->")
    
    # BeautifulSoup zum Parsen nutzen Image
    soup = BeautifulSoup(comm.sub("",res.text),'html')
    data = soup.find_all("div", )[0]
    data = data.find_all("section")[0]
    img = data.find_all("div", class_ = "character-portrait")[0].img["src"]
    filename = os.path.basename(img)  # "executioner.png"
    os.makedirs(download_folder, exist_ok=True)
    filepath = os.path.join(download_folder, filename)
    
    # Datei herunterladen
    response = requests.get(img)
    if response.status_code == 200:
        with open(filepath, "wb") as file:
            file.write(response.content)
        print(f"Heruntergeladen: {filename}")
    else:
        print(f"Fehler beim Herunterladen: {url}")
    img = str(filename)
    name = data.find_all("div", class_ = "character-portrait")[0].img["alt"]
    cost_li = data.find_all("ul")[0].li
    cost_value = int(cost_li.get_text(strip=True).replace("Cost:", "").strip())
    # traits
    soup = BeautifulSoup(comm.sub("",res.text),'html')
    data = soup.find_all("div", )[0]
    data = data.find_all("section")[0]
    data = data.find_all("div")[1]
    traits = data.find_all("div", class_ = "character-ability")
    traits_list = []
    for i in range(1, len(traits)):
        trait = traits[i].img["alt"]
        traits_list.append(trait)
    
    print(f"{name}, {img}, {cost_value}, {traits_list}")
    champ = {
        'name': name,
        'cost': cost_value,
        'traits': traits_list,
        'tier': cost_value
    }
    return champ

In [52]:
url = "https://tftactics.gg/set-update/"
res = requests.get(url)
comm = re.compile("<!--|-->")

# BeautifulSoup zum Parsen nutzen
soup = BeautifulSoup(comm.sub("",res.text),'html')
data = soup.find_all("div", class_ = "set-traits")[0]
trait_data = data.find_all("div", class_ = "trait-champions-list")
trait_data[0].find_all("a")[0]["href"]
# Daten enthalten nun alle Traits mit den einzelnen champs
champ_list = []
for trait in trait_data:
    trait_champions = trait.find_all("a")
    for i in range(len(trait_champions)):
        champ_ref = trait_champions[i]["href"]
        print(champ_ref)
        champ = get_champ_dict(champ_ref)
        champ_list.append(champ)

# Da aber champs mehrere traits haben, kommen die deshalb doppelt vor, lösche die duplikate
unique_list = []
seen = set()

for champ in champ_list:
    # Konvertiere alle Werte zu Tupeln, falls sie Listen sind
    champ_tuple = tuple((k, tuple(v) if isinstance(v, list) else v) for k, v in champ.items())

    if champ_tuple not in seen:
        seen.add(champ_tuple)
        unique_list.append(champ)

len(unique_list)

/champions/aurora/
Heruntergeladen: Aurora.png
Aurora, Aurora.png, 5, ['Anima Squad', 'Dynamo']
/champions/illaoi/
Heruntergeladen: Illaoi.png
Illaoi, Illaoi.png, 2, ['Anima Squad', 'Bastion']
/champions/leona/
Heruntergeladen: Leona.png
Leona, Leona.png, 4, ['Anima Squad', 'Vanguard']
/champions/seraphine/
Heruntergeladen: Seraphine.png
Seraphine, Seraphine.png, 1, ['Anima Squad', 'Techie']
/champions/sylas/
Heruntergeladen: Sylas.png
Sylas, Sylas.png, 1, ['Anima Squad', 'Vanguard']
/champions/vayne/
Heruntergeladen: Vayne.png
Vayne, Vayne.png, 2, ['Anima Squad', 'Slayer']
/champions/xayah/
Heruntergeladen: Xayah.png
Xayah, Xayah.png, 4, ['Anima Squad', 'Marksman']
/champions/yuumi/
Heruntergeladen: Yuumi.png
Yuumi, Yuumi.png, 3, ['Anima Squad', 'A.M.P.', 'Strategist']
/champions/chogath/
Heruntergeladen: Chogath.png
Chogath, Chogath.png, 4, ['BoomBots', 'Bruiser']
/champions/fiddlesticks/
Heruntergeladen: Fiddlesticks.png
Fiddlesticks, Fiddlesticks.png, 3, ['BoomBots', 'Techie']
/cha

60

In [24]:
unique_list

[{'name': 'Aurora', 'cost': 5, 'traits': ['Anima Squad', 'Dynamo'], 'tier': 5},
 {'name': 'Illaoi',
  'cost': 2,
  'traits': ['Anima Squad', 'Bastion'],
  'tier': 2},
 {'name': 'Leona',
  'cost': 4,
  'traits': ['Anima Squad', 'Vanguard'],
  'tier': 4},
 {'name': 'Seraphine',
  'cost': 1,
  'traits': ['Anima Squad', 'Techie'],
  'tier': 1},
 {'name': 'Sylas',
  'cost': 1,
  'traits': ['Anima Squad', 'Vanguard'],
  'tier': 1},
 {'name': 'Vayne', 'cost': 2, 'traits': ['Anima Squad', 'Slayer'], 'tier': 2},
 {'name': 'Xayah',
  'cost': 4,
  'traits': ['Anima Squad', 'Marksman'],
  'tier': 4},
 {'name': 'Yuumi',
  'cost': 3,
  'traits': ['Anima Squad', 'A.M.P.', 'Strategist'],
  'tier': 3},
 {'name': 'Chogath', 'cost': 4, 'traits': ['BoomBots', 'Bruiser'], 'tier': 4},
 {'name': 'Fiddlesticks',
  'cost': 3,
  'traits': ['BoomBots', 'Techie'],
  'tier': 3},
 {'name': 'Kogmaw', 'cost': 1, 'traits': ['BoomBots', 'Rapidfire'], 'tier': 1},
 {'name': 'Skarner', 'cost': 2, 'traits': ['BoomBots', 'V

# Traits and Origins

In [10]:
def get_trait_pngs(champ_ref, trait_list):
    # übergebe die ref und die leere trait_list für pngs
    url = f"https://tftactics.gg{champ_ref}/"
    res = requests.get(url)
    comm = re.compile("<!--|-->")
    # traits
    soup = BeautifulSoup(comm.sub("",res.text),'html')
    data = soup.find_all("div", )[0]
    data = data.find_all("section")[0]
    data = data.find_all("div")[1]
    traits = data.find_all("div", class_ = "character-ability")
    traits_list = []
    for i in range(1, len(traits)):
        trait = traits[i].img["src"]
        trait_list.append(trait)
    
    return trait_list

In [12]:
url = "https://tftactics.gg/set-update/"
res = requests.get(url)
comm = re.compile("<!--|-->")

# BeautifulSoup zum Parsen nutzen
soup = BeautifulSoup(comm.sub("",res.text),'html')
data = soup.find_all("div", class_ = "set-traits")[0]
trait_data = data.find_all("div", class_ = "trait-champions-list")
trait_data[0].find_all("a")[0]["href"]
# Daten enthalten nun alle Traits mit den einzelnen champs
trait_png_list = []
for trait in trait_data:
    trait_champions = trait.find_all("a")
    for i in range(len(trait_champions)):
        champ_ref = trait_champions[i]["href"]
        get_trait_pngs(champ_ref, trait_png_list)

trait_png_list = list(set(trait_png_list))
trait_png_list = list(dict.fromkeys(trait_png_list))

download_folder = "downloaded_origins"
os.makedirs(download_folder, exist_ok=True)

# Bilder herunterladen
for url in trait_png_list:
    filename = os.path.basename(url).capitalize()  # "executioner.png"
    filepath = os.path.join(download_folder, filename)
    
    # Datei herunterladen
    response = requests.get(url)
    if response.status_code == 200:
        with open(filepath, "wb") as file:
            file.write(response.content)
        print(f"Heruntergeladen: {filename}")
    else:
        print(f"Fehler beim Herunterladen: {url}")

Heruntergeladen: Rapidfire.png
Heruntergeladen: A.m.p..png
Heruntergeladen: Vanguard.png
Heruntergeladen: Exotech.png
Heruntergeladen: Cyberboss.png
Heruntergeladen: Nitro.png
Heruntergeladen: Bastion.png
Heruntergeladen: Executioner.png
Heruntergeladen: Syndicate.png
Heruntergeladen: Techie.png
Heruntergeladen: Overlord.png
Heruntergeladen: Godofthenet.png
Heruntergeladen: Strategist.png
Heruntergeladen: Boombots.png
Heruntergeladen: Streetdemon.png
Heruntergeladen: Cypher.png
Heruntergeladen: Virus.png
Heruntergeladen: Goldenox.png
Heruntergeladen: Animasquad.png
Heruntergeladen: Bruiser.png
Heruntergeladen: Marksman.png
Heruntergeladen: Soulkiller.png
Heruntergeladen: Dynamo.png
Heruntergeladen: Slayer.png
Heruntergeladen: Divinicorp.png


In [38]:
download_folder = "downloaded_images"
url = f"https://tftactics.gg/champions/aurora//"
res = requests.get(url)
comm = re.compile("<!--|-->")

# BeautifulSoup zum Parsen nutzen Image
soup = BeautifulSoup(comm.sub("",res.text),'html')
data = soup.find_all("div", )[0]
data = data.find_all("section")[0]
img = data.find_all("div", class_ = "character-portrait")[0].img["src"]
filename = os.path.basename(img)  # "executioner.png"
os.makedirs(download_folder, exist_ok=True)
filepath = os.path.join(download_folder, filename)



In [42]:
img

'https://rerollcdn.com/characters/Skin/14/Aurora.png'

In [ ]:
# Datei herunterladen
response = requests.get(url)
if response.status_code == 200:
    with open(filepath, "wb") as file:
        file.write(response.content)
    print(f"Heruntergeladen: {filename}")
else:
    print(f"Fehler beim Herunterladen: {url}")
img = str(filename)
name = data.find_all("div", class_ = "character-portrait")[0].img["alt"]
cost_li = data.find_all("ul")[0].li
cost_value = int(cost_li.get_text(strip=True).replace("Cost:", "").strip())